<a href="https://colab.research.google.com/github/newmantic/simulated_annealing/blob/main/simulated_annealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def simulated_annealing(objective_function, bounds, max_iter, initial_temp, cooling_rate):
    """
    Perform Simulated Annealing to minimize the given objective function.

    :param objective_function: The objective function to minimize.
    :param bounds: A list of tuples specifying the lower and upper bounds for each dimension.
    :param max_iter: Maximum number of iterations to run the algorithm.
    :param initial_temp: The initial temperature for the annealing process.
    :param cooling_rate: The rate at which the temperature decreases.
    :return: The best solution found and its objective function value.
    """
    # Initialize the starting point randomly within the bounds
    current_solution = np.array([np.random.uniform(low, high) for low, high in bounds])
    current_value = objective_function(current_solution)

    # Initialize the best solution as the current one
    best_solution = np.copy(current_solution)
    best_value = current_value

    # Initialize the current temperature
    temperature = initial_temp

    for i in range(max_iter):
        # Generate a new candidate solution by perturbing the current solution
        candidate_solution = current_solution + np.random.normal(0, 1, size=current_solution.shape)

        # Clip the candidate solution to respect the bounds
        candidate_solution = np.clip(candidate_solution, [low for low, _ in bounds], [high for _, high in bounds])

        # Evaluate the candidate solution
        candidate_value = objective_function(candidate_solution)

        # Calculate the acceptance probability
        delta = candidate_value - current_value
        acceptance_probability = np.exp(-delta / temperature) if delta > 0 else 1.0

        # Decide whether to accept the candidate solution
        if np.random.rand() < acceptance_probability:
            current_solution = candidate_solution
            current_value = candidate_value

            # Update the best solution if the candidate is better
            if candidate_value < best_value:
                best_solution = candidate_solution
                best_value = candidate_value

        # Cool down the temperature
        temperature *= cooling_rate

    return best_solution, best_value

In [2]:
def rastrigin_function(x):
    n = len(x)
    return 10 * n + sum([xi**2 - 10 * np.cos(2 * np.pi * xi) for xi in x])

def test_case_1():
    bounds = [(-5.12, 5.12)] * 2  # 2-dimensional problem
    max_iter = 1000
    initial_temp = 100
    cooling_rate = 0.95

    best_solution, best_value = simulated_annealing(rastrigin_function, bounds, max_iter, initial_temp, cooling_rate)
    print(f"Best solution: {best_solution}")
    print(f"Best objective value: {best_value}")
    # Expected: The best value should be close to 0, the global minimum of the Rastrigin function.

test_case_1()

Best solution: [0.00882339 0.01161775]
Best objective value: 0.04220689904783015


In [3]:
def ackley_function(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    n = len(x)
    sum1 = np.sum(np.square(x))
    sum2 = np.sum(np.cos(c * x))
    return -a * np.exp(-b * np.sqrt(sum1 / n)) - np.exp(sum2 / n) + a + np.exp(1)

def test_case_2():
    bounds = [(-5, 5)] * 2  # 2-dimensional problem
    max_iter = 1000
    initial_temp = 100
    cooling_rate = 0.95

    best_solution, best_value = simulated_annealing(ackley_function, bounds, max_iter, initial_temp, cooling_rate)
    print(f"Best solution: {best_solution}")
    print(f"Best objective value: {best_value}")
    # Expected: The best value should be close to 0, the global minimum of the Ackley function.

test_case_2()

Best solution: [ 0.00586967 -0.01892723]
Best objective value: 0.06647454588487678


In [4]:
def distance_matrix(cities):
    n = len(cities)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist_matrix[i, j] = np.linalg.norm(cities[i] - cities[j])
            dist_matrix[j, i] = dist_matrix[i, j]
    return dist_matrix

def tsp_objective_function(permutation, dist_matrix):
    return sum(dist_matrix[permutation[i-1], permutation[i]] for i in range(len(permutation)))

def test_case_3():
    cities = np.array([
        [0, 0],
        [1, 2],
        [2, 4],
        [3, 6],
        [4, 1],
        [5, 3],
        [6, 5],
        [7, 0]
    ])

    dist_matrix = distance_matrix(cities)
    bounds = [(0, len(cities)-1)] * len(cities)  # Permutation bounds

    def perm_objective(perm):
        return tsp_objective_function(np.argsort(perm), dist_matrix)

    max_iter = 1000
    initial_temp = 1000
    cooling_rate = 0.995

    best_solution, best_value = simulated_annealing(perm_objective, bounds, max_iter, initial_temp, cooling_rate)
    best_tour = np.argsort(best_solution)

    print(f"Best tour: {best_tour}")
    print(f"Best tour length: {best_value}")
    # Expected: The best tour length should be minimal for the given set of cities.

test_case_3()

Best tour: [7 4 1 0 2 3 6 5]
Best tour length: 24.272724143468075
